In [1]:
import os, sys, time, shutil
import warnings
import numpy as np
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
pd.options.mode.chained_assignment = None
import h5py

%matplotlib inline

from IPython.display import display, HTML

if (sys.version_info < (3, 0)):
    HTML('''<script>
    code_show=true; 
    function code_toggle() {
     if (code_show){
     $('div.input').hide();
     } else {
     $('div.input').show();
     }
     code_show = !code_show
    } 
    $( document ).ready(code_toggle);
    </script>
    <form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [2]:
CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

# Relative path between notebooks and goruped output directories
relative_path = '../../../outputs'
survey_path = '../../../inputs/base_year/survey'

In [3]:
pd.options.display.float_format = '{:0,.0f}'.format

In [4]:
def load_network_summary(filepath):
    """Load network-level results using a standard procedure. """
    df = pd.read_csv(filepath)

    # Congested network components by time of day
    df.columns

    # Get freeflow from 20to5 period

    # Exclude trips taken on non-designated facilities (facility_type == 0)
    # These are artificial (weave lanes to connect HOV) or for non-auto uses 
    df = df[df['data3'] != 0]    # data3 represents facility_type

    # calculate total link VMT and VHT
    df['VMT'] = df['@tveh']*df['length']
    df['VHT'] = df['@tveh']*df['auto_time']/60

    # Define facility type
    df.loc[df['data3'].isin([1,2]), 'facility_type'] = 'highway'
    df.loc[df['data3'].isin([3,4,6]), 'facility_type'] = 'arterial'
    df.loc[df['data3'].isin([5]), 'facility_type'] = 'connector'

    # Calculate delay
    # Select links from overnight time of day
    delay_df = df.loc[df['tod'] == '20to5'][['ij','auto_time']]
    delay_df.rename(columns={'auto_time':'freeflow_time'}, inplace=True)

    # Merge delay field back onto network link df
    df = pd.merge(df, delay_df, on='ij', how='left')

    # Calcualte hourly delay
    df['total_delay'] = ((df['auto_time']-df['freeflow_time'])*df['@tveh'])/60    # sum of (volume)*(travtime diff from freeflow)

    df['county'] =df['@countyid'].map({33: 'King',
                                      35: 'Kitsap',
                                      53: 'Pierce',
                                      61: 'Snohomish'})
    df['county'].fillna('Outside Region', inplace=True)
    
    return df

In [5]:
# Create outputs dir 
dir = r'..\..\..\outputs\compare\RTP\standard'
if os.path.exists(dir):
    shutil.rmtree(dir)
os.makedirs(dir)

# Households and Population

In [6]:
# Total Population
# Load Network Summary sheet from network_summary_detailed
df = pd.read_csv(os.path.join(relative_path,'agg/dash',r'pptyp_county.csv'))
df = df.groupby('person_county').sum()[['psexpfac']]
df.loc['Total',:] = df.sum()
df = df.reset_index()
df.rename(columns={'psexpfac': 'Population', 'person_county': 'County'}, inplace=True)
df_hh = pd.read_csv(os.path.join(relative_path,'agg/dash',r'hh_geog.csv'))
df_hh = df_hh.groupby('hh_county').sum()[['hhexpfac']]
df_hh.loc['Total',:] = df_hh.sum()
df_hh = df_hh.reset_index()
df_hh.rename(columns={'hhexpfac': 'Households', 'hh_county': 'County'}, inplace=True)

df = df_hh.merge(df, on='County')
df.to_csv(os.path.join(os.getcwd(), r'..\..\..\outputs\compare\RTP\standard','1_total_population.csv'),
         index=False)
HTML(df.to_html(index=False))

County,Households,Population
King,"866,598","2,052,920"
Kitsap,"39,114","93,942"
Pierce,"196,988","490,444"
Snohomish,"187,282","481,038"
Total,"1,289,982","3,118,344"


# Employment
By Workplace Location

In [7]:
df = pd.read_csv(os.path.join(relative_path,'agg/dash',r'person_worker_type.csv'))
df = df.groupby('person_work_county').sum()[['psexpfac']]
df.loc['Total',:] = df.sum()
df = df.reset_index()
df.rename(columns={'psexpfac': 'Population', 'person_work_county': 'County'}, inplace=True)
df.to_csv(os.path.join(os.getcwd(), r'..\..\..\outputs\compare\RTP\standard','2_total_employment.csv'))
HTML(df.to_html(index=False))

County,Population
King,"1,045,929"
Kitsap,"18,203"
Pierce,"153,267"
Snohomish,"134,464"
Total,"1,351,863"


# Vehicle Miles Traveled
Includes truck and external trips

In [8]:
df_network = load_network_summary(os.path.join(relative_path,'network',r'network_results.csv'))
df = df_network.groupby('county').sum()[['VMT']]
df.loc['Total',:] = df.sum()
df = df.reset_index()
df.rename(columns={'county':'County'}, inplace=True)
sort_map = {'King':1,'Kitsap':2,'Pierce':3,'Snohomish':4,'Outside Region':5,'Total':6}
df['sort'] = df['County'].map(sort_map)
df = df.sort_values('sort')
df.drop('sort', axis=1, inplace=True)
df.to_csv(os.path.join(os.getcwd(), r'..\..\..\outputs\compare\RTP\standard','3_vmt.csv'))
HTML(df.to_html(index=False))

County,VMT
King,"45,054,578"
Kitsap,"4,293,918"
Pierce,"17,971,135"
Snohomish,"16,539,940"
Outside Region,"256,871"
Total,"84,116,442"


# Vehicle Hours Traveled

In [9]:
df = df_network.groupby('county').sum()[['VHT']]
df.loc['Total',:] = df.sum()
df = df.reset_index()
df.rename(columns={'county':'County'}, inplace=True)
sort_map = {'King':1,'Kitsap':2,'Pierce':3,'Snohomish':4,'Outside Region':5,'Total':6}
df['sort'] = df['County'].map(sort_map)
df = df.sort_values('sort')
df.drop('sort', axis=1, inplace=True)
df.to_csv(os.path.join(os.getcwd(), r'..\..\..\outputs\compare\RTP\standard','4_vht.csv'))
HTML(df.to_html(index=False))

County,VHT
King,"1,448,799"
Kitsap,"125,097"
Pierce,"548,075"
Snohomish,"497,155"
Outside Region,"5,010"
Total,"2,624,136"


# Vehicle Hours of Delay

In [10]:
df = df_network.groupby('county').sum()[['total_delay']]
df.rename(columns={'total_delay': 'Total Delay Hours'}, inplace=True)
df.loc['Total',:] = df.sum()
df = df.reset_index()
df.rename(columns={'county':'County'}, inplace=True)
sort_map = {'King':1,'Kitsap':2,'Pierce':3,'Snohomish':4,'Outside Region':5,'Total':6}
df['sort'] = df['County'].map(sort_map)
df = df.sort_values('sort')
df.drop('sort', axis=1, inplace=True)
df.to_csv(os.path.join(os.getcwd(), r'..\..\..\outputs\compare\RTP\standard','5_delay.csv'))
HTML(df.to_html(index=False))

County,Total Delay Hours
King,"194,412"
Kitsap,"2,859"
Pierce,"44,284"
Snohomish,"42,947"
Outside Region,6
Total,"284,509"


# Transit Boardings

In [11]:
df = pd.read_csv(os.path.join(relative_path,'transit',r'daily_boardings_by_agency.csv'), index_col='agency_name')
df.loc['Total',:] = df['boardings'].sum()
df = df.reset_index()
df.rename(columns={'agency_name':'Agency', 'boardings': 'Daily Boardings'}, inplace=True)
df.to_csv(os.path.join(os.getcwd(), r'..\..\..\outputs\compare\RTP\standard','6_transit_boardings.csv'))
HTML(df.to_html(index=False))

Agency,Daily Boardings
King County Metro,"338,507"
Sound Transit,"130,893"
Community Transit,"23,673"
Pierce Transit,"17,491"
Kitsap Transit,"16,540"
Washington Ferries,"8,085"
Everett Transit,"2,723"
Total,"537,911"


# Mode Share
All Trips

In [12]:
pd.options.display.float_format = '{:0,.1%}'.format
df_trip = pd.read_csv(r'..\..\..\outputs\agg\dash\mode_share_county.csv')
df = df_trip.groupby('mode').sum()[['trexpfac']]/df_trip['trexpfac'].sum()
df = df.reset_index()
df.rename(columns={'mode': 'Mode', 'trexpfac': 'Mode Share'}, inplace=True)
df.to_csv(os.path.join(os.getcwd(), r'..\..\..\outputs\compare\RTP\standard','7_mode_share.csv'))
HTML(df.to_html(index=False))

Mode,Mode Share
Bike,1.4%
HOV2,20.3%
HOV3+,17.1%
SOV,44.3%
School Bus,1.8%
TNC,0.2%
Transit,2.8%
Walk,12.1%


# Emissions
Daily Tons for light, medium, and heavy vehicles; bus vehicles are excluded.

In [13]:
pd.options.display.float_format = '{:0,.0f}'.format
df = pd.read_csv(os.path.join(relative_path,'emissions',r'emissions_summary.csv'))
df = df[df['veh_type'].isin(['light','medium','heavy'])]
df = df.groupby('pollutant_name').sum()
df.rename(columns={'start_tons': 'Start', 'intrazonal_tons': 'Intrazonal', 'interzonal_tons': 'Interzonal',
                  'total_daily_tons': 'Total Daily'}, inplace=True)
df = df[df.index.isin(['CO','NOx','PM25 Total','PM10 Total','CO2 Equivalent'])]
df = df.applymap(lambda x: x if x > 100 else str(round(x,1)))
df.to_csv(os.path.join(os.getcwd(), r'..\..\..\outputs\compare\RTP\standard','8_emissions.csv'))
df

,Start,Intrazonal,Interzonal,Total Daily
pollutant_name,,,,
CO,195,2.9,288,486
CO2 Equivalent,"2,693",308,"39,474","42,475"
NOx,11.1,0.3,68.6,80.1
PM10 Total,0.5,0.1,5.6,6.1
PM25 Total,0.4,0.0,1.8,2.2


# Lane Miles

In [14]:
pd.options.display.float_format = '{:0,.0f}'.format
df_network = load_network_summary(os.path.join(relative_path,'network',r'network_results.csv'))
# Select mid-day network
gdf = df_network[df_network['tod'] == '10to14']
gdf['Lane Miles'] = gdf['length']*gdf['num_lanes']

In [15]:
facility_type_dict = {
    1: 'Interstate',
    2: 'Other Freeway',
    3: 'Expressway',
    4: 'Ramp',
    5: 'Principal Arterial',
    6: 'Minor Arterial',
    7: 'Major Collector',
    8: 'Minor Collector',
    9: 'Local',
    10: 'Busway',
    11: 'Non-Motor',
    12: 'Light Rail',
    13: 'Commuter Rail',
    15: 'Ferry',
    16: 'Passenger-Only Ferry',
    17: 'Centroid Connector',
    18: 'Facility Connector',
    19: 'HOV Only Freeway',
    20: 'HOV Only Ramp',
    98: 'Weave Links'
}

ul3_dict = {
    0: 'Rail/Walk/Ferry',
    1: 'Freeway',
    2: 'Expressway',
    3: 'Urban Arterial',
    4: 'One-way Arterial',
    5: 'Centroid Connector',
    6: 'Rural Arterial'
}

county_dict = {
    33: 'King',
    35: 'Kitsap',
    53: 'Pierce',
    61: 'Snohomish'
}

In [16]:
gdf['Facility Type'] = gdf['@facilitytype'].map(facility_type_dict)
df = gdf.groupby(['Facility Type','@facilitytype']).sum()[['Lane Miles']].sort_values('@facilitytype').reset_index()
df.index = df['Facility Type']
df = df.drop(['@facilitytype','Facility Type'], axis=1)
df.loc['Total','Lane Miles'] = df['Lane Miles'].sum()
df

,Lane Miles
Facility Type,
Interstate,"1,468"
Other Freeway,767
Expressway,229
Ramp,538
Principal Arterial,"3,101"
Minor Arterial,"4,066"
Major Collector,"3,033"
Minor Collector,224
Local,318


In [17]:
gdf['Facility Group'] = gdf['data3'].map(ul3_dict)
df = gdf.groupby(['Facility Group','data3']).sum()[['Lane Miles']].sort_values('data3').reset_index()
df.index = df['Facility Group']
df = df.drop(['data3','Facility Group'], axis=1)
df.loc['Total','Lane Miles'] = df['Lane Miles'].sum()
df

,Lane Miles
Facility Group,
Freeway,"2,240"
Expressway,771
Urban Arterial,"6,888"
One-way Arterial,440
Centroid Connector,"10,267"
Rural Arterial,"3,428"
Total,"24,033"


In [18]:
# Load link attributes and join
gdf['county'].replace(np.NaN,'Outside Region',inplace=True)
df = gdf.groupby(['county']).sum()[['Lane Miles']].reset_index()
df['order'] = df['county'].map({'King': 1, 'Kitsap': 2, 'Pierce': 3, 'Snohomish': 4, 'Outside Region': 5})
df = df.sort_values('order').drop('order', axis=1)
df.index = df['county']
df = df.drop('county', axis=1)
df.loc['Total','Lane Miles'] = df['Lane Miles'].sum()
df

,Lane Miles
county,
King,"10,944"
Kitsap,"2,175"
Pierce,"6,012"
Snohomish,"4,856"
Outside Region,59
Total,"24,046"
